# Deep learning models

### Todo



#### This cell is for defining various OPTIONS used for this notebook (working directory, how many rows and columns pandas displays for a dataframe, etc). 

#### Preferably this cell is also where we do important imports (for example pandas and numpy)

In [294]:
import os 
#Input the directory where your joined_data.csv is located 
#os.chdir('C:/Users/Trond/Documents/Master 2020/Processed data')
os.chdir('C:/Users/Briggstone/Documents/Master 2020/Processed data')
#os.chdir('C:/Users/MyPC/Documents/Andrijana/UiS/DATMAS Master oppgave/Processed data')

#Where you want the csv file of the merged data to be placed
output_filepath = 'C:/Users/Briggstone/Documents/Master 2020/Processed data'
#output_filepath = 'C:/Users/MyPC/Documents/Andrijana/UiS/DATMAS Master oppgave/Processed data'

import numpy as np
import pandas as pd

# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

# Which folder of preprocessed data we want to use
FOLDER_NAME = "LSTM_TEST"


#### In this cell we do additional preprocessing specific for LSTM

In [295]:
def create_generator_arrays(df):
    
    df_x = []
    df_y = []
    
    unique_subjects = df.PATNO.unique()  
    
    for u in unique_subjects:
        
        subject_data = df.loc[df.PATNO == u, :]
        
        y = subject_data.tail(1).HALL.values[0]
        y = np.expand_dims(y, axis = 0)
        df_y.append(y)
        
        subject_data.drop(["HALL", "PATNO"], axis = 1, inplace = True)
        
        x = subject_data.to_numpy()
        x = np.expand_dims(x, axis=0)
        df_x.append(x)
        
    return df_x,df_y




train = pd.read_csv(FOLDER_NAME + '/train.csv')
test = pd.read_csv(FOLDER_NAME + '/test.csv')

train.drop(["SAMPLING"], axis = 1, inplace  = True)

train_x, train_y = create_generator_arrays(train)
test_x, test_y = create_generator_arrays(test)


num_of_dims = train.shape[1] - 2
num_of_unique_subjects_train = train.PATNO.unique().size  
num_of_unique_subjects_test = test.PATNO.unique().size  

C:\Users\Briggstone\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [296]:
sum(train_y) / len(train_y)

array([0.25625])

#### In this section we define helper functions for the LSTM implementation

In [297]:
def data_generator (x,y, num_of_epochs):
    for e in range(num_of_epochs):
        for i in range (len(x)):
            yield x[i], y[i]
        

In [298]:
type(train_y[0])

numpy.ndarray

#### In this cell we set all flags and parameters for LSTM (Long short-term memory)

#### In this cell we implement our LSTM

In [299]:
from keras.models import Sequential 
from keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(4, input_shape=(None, num_of_dims)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])


from sklearn.utils import class_weight

train_y_temp = [y.item() for y in train_y]

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_y_temp),
                                                 train_y_temp)


model.fit_generator(data_generator(train_x,train_y, 60), steps_per_epoch = num_of_unique_subjects_train, epochs = 60, verbose = 1, class_weight=class_weights)


Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_72 (LSTM)               (None, 4)                 624       
_________________________________________________________________
dropout_26 (Dropout)         (None, 4)                 0         
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 5         
Total params: 629
Trainable params: 629
Non-trainable params: 0
_________________________________________________________________
Epoch 1/60
320/320 [==============================] - 1s 2ms/step - loss: 0.5973 - accuracy: 0.7406
Epoch 2/60
320/320 [==============================] - 1s 2ms/step - loss: 0.5656 - accuracy: 0.7437
Epoch 3/60
320/320 [==============================] - 1s 2ms/step - loss: 0.5752 - accuracy: 0.7437
Epoch 4/60
320/320 [==============================] - 1s 2ms/step - loss: 0.5518 - ac

In [300]:
predictions = model.predict_generator(data_generator(test_x, test_y, 1), steps = num_of_unique_subjects_test) > 0.5

In [301]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_y, predictions)

array([[54,  7],
       [16,  4]], dtype=int64)

In [302]:
from sklearn import metrics as met

print(met.accuracy_score(test_y, predictions))
print(met.recall_score(test_y, predictions))
print(met.f1_score(test_y, predictions))
print(met.roc_auc_score(test_y, predictions))

0.7160493827160493
0.2
0.25806451612903225
0.542622950819672
